In [1]:
# pip install langchain-mcp-adapters
!ls

README.md               resources               test_mcp_notebook.ipynb
__pycache__             sqlite_server           test_servers
model_instance.py       streamlit_app.py
requirements.txt        test.py


In [2]:
# from test_servers.client import main
from sqlite_server.client import main

In [ ]:
question = "what is the total number of artists ?"

question = "List top 10 tracks with their corresponding artist and album names."

response= await main(question)
print(response["messages"][-1].content)

Here are the top 10 tracks along with their corresponding artist and album names:

1. **Track:** For Those About To Rock (We Salute You)  
   **Album:** For Those About To Rock We Salute You  
   **Artist:** AC/DC

2. **Track:** Balls to the Wall  
   **Album:** Balls to the Wall  
   **Artist:** Accept

3. **Track:** Fast As a Shark  
   **Album:** Restless and Wild  
   **Artist:** Accept

4. **Track:** Restless and Wild  
   **Album:** Restless and Wild  
   **Artist:** Accept

5. **Track:** Princess of the Dawn  
   **Album:** Restless and Wild  
   **Artist:** Accept

6. **Track:** Put The Finger On You  
   **Album:** For Those About To Rock We Salute You  
   **Artist:** AC/DC

7. **Track:** Let's Get It Up  
   **Album:** For Those About To Rock We Salute You  
   **Artist:** AC/DC

8. **Track:** Inject The Venom  
   **Album:** For Those About To Rock We Salute You  
   **Artist:** AC/DC

9. **Track:** Snowballed  
   **Album:** For Those About To Rock We Salute You  
   **Art

In [17]:
# from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, FunctionMessage
# from pprint import pprint

# for i, message in enumerate(response["messages"]):
#     print(f"\nMessage {i + 1} - Type: {type(message).__name__}")
#     print("-" * 50)

#     if isinstance(message, HumanMessage):
#         print(f"User Input: {message.content}")

#     elif isinstance(message, AIMessage):
#         print(f"AI Response: {message.content}")
#         if message.tool_calls:
#             print("Tool Calls:")
#             for tool_call in message.tool_calls:
#                 print(f"  Tool Name: {tool_call['name']}")
#                 print("  Arguments:")
#                 pprint(tool_call['args'])

#     elif isinstance(message, ToolMessage):
#         print(f"Tool Response from {message.tool_call_id}:")
#         print(f"  Content: {message.content}")

#     elif isinstance(message, FunctionMessage):
#         print(f"Function Response:")
#         print(f"  Name: {message.name}")
#         print(f"  Content: {message.content}")

#     else:
#         print("Other Message:")
#         pprint(message)

#     print("-*-" * 60)

In [19]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, FunctionMessage

def format_response(response):
    """
    Format the response from the server for better readability.
    """

    formatted_response = ""
    
    for i, message in enumerate(response["messages"]):
        formatted_response += "\n" + f"\nMessage {i + 1} - Type: {type(message).__name__}"
        formatted_response += "\n" + "-" * 50

        if isinstance(message, HumanMessage):
            formatted_response += "\n" + f"User Input: {message.content}"

        elif isinstance(message, AIMessage):
            formatted_response += "\n" + f"AI Response: {message.content}"
            if message.tool_calls:
                formatted_response += "\n" + "Tool Calls:"
                for tool_call in message.tool_calls:
                    formatted_response += "\n" + f"  Tool Name: {tool_call['name']}"
                    formatted_response += "\n" + "  Arguments:"
                    formatted_response += "\n" + str(tool_call['args'])

        elif isinstance(message, ToolMessage):
            formatted_response += "\n" + f"Tool Response from {message.tool_call_id}:"
            formatted_response += "\n" + f"  Content: {message.content}"

        elif isinstance(message, FunctionMessage):
            formatted_response += "\n" + f"Function Response:"
            formatted_response += "\n" + f"  Name: {message.name}"
            formatted_response += "\n" + f"  Content: {message.content}"

        else:
            formatted_response += "\n" + "Other Message:"
            formatted_response += "\n" + message

        formatted_response += "\n" + "-*-" * 60

    return formatted_response


formatted_response = format_response(response)

In [20]:
print(formatted_response)



Message 1 - Type: HumanMessage
--------------------------------------------------
User Input: List top 10 tracks with their corresponding artist and album names.
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

Message 2 - Type: AIMessage
--------------------------------------------------
AI Response: 
Tool Calls:
  Tool Name: list_tables
  Arguments:
{}
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

Message 3 - Type: ToolMessage
--------------------------------------------------
Tool Response from call_Sim7ooSsW4dfNzazLstPuKFR:
  Content: [{'name': 'albums'}, {'name': 'sqlite_sequence'}, {'name': 'artists'}, {'name': 'customers'}, {'name': 'employees'}, {'name': 'genres'}, {'name': 'invoices'}, {'name': 'invoice_items'}

In [15]:
import sqlite3
import pandas as pd

def get_all_tables(db_path: str) -> list:
    """
    Returns a list of all table names in the SQLite database.
    
    Parameters:
        db_path (str): Path to the .db file

    Returns:
        list: List of table names
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]

        conn.close()
        return tables

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return []


def execute_query(db_path: str, query: str) -> pd.DataFrame:
    """
    Executes a SQL query and returns the result as a pandas DataFrame.
    
    Parameters:
        db_path (str): Path to the .db file
        query (str): SQL query to execute

    Returns:
        pd.DataFrame: Query results as a DataFrame
    """
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df

    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()



In [8]:
db_path = "sqlite_server/sql_db.db"

all_tables = get_all_tables(db_path)

all_tables

['albums',
 'sqlite_sequence',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks',
 'sqlite_stat1']

In [16]:
query = "select * from employees limit 10;"

df = execute_query(db_path, query)

df

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
